In [30]:
import numpy as np

class NN():
    def __init__(self, input_layer_size = 2, hidden_layer_size = 2, output_layer_size = 2, hidden_layers = 1, lr = 0.1, epoches = 1000):
        self.input_layer_size = input_layer_size
        self.hidden_layer_size = hidden_layer_size
        self.output_layer_size = output_layer_size
        self.hidden_layers = hidden_layers
        self.lr = lr
        self.number_of_epoches = epoches

        # initialize the weights by random numbers
        self.W = [np.random.randn(self.input_layer_size, self.hidden_layer_size)]
        for layer in range(hidden_layers-1):
            self.W.append(np.random.randn(self.hidden_layer_size, self.hidden_layer_size))
        self.W.append(np.random.randn(self.hidden_layer_size, self.output_layer_size))

    def activation(self, x): # sigmoid function
        return (1/(1+np.exp(-x)))
    
    def forward(self, X, less_steps = False): # forward prapogation
        # X- input
        input_list = X

        if less_steps == False: # for full forwward propagation to get final output
            less_steps = self.hidden_layers + 1 
        # if a value is given to "steps" parameter, then we perform forward propagation to that step 

        for step in range(less_steps):
            output_list = np.dot(input_list, self.W[step])
            output_list = [self.activation(neuron) for neuron in output_list]
            input_list = output_list
        return input_list # since input_list = output_list at the end of the iterations and also we can use it for our loop in backward()
    
    def backward(self, X, y): # backward propagation
        # X- input, y- output
        # output_list = self.forward(X)
        # total error = np.sum((y - output_list)**2/2)
        hidden_layers = self.hidden_layers
        # first back propagation for last layer
        # error = (output)(1-output)(actual-output)
        layer_error = np.array(self.forward(X)) * (1 - np.array(self.forward(X))) * (np.array(y) - np.array(self.forward(X)))
        # Weight updation- W = W + alpha * (H * ((y-OP) * (OP * (1 - OP))))
        self.W[-1] += self.lr * (np.dot(np.matrix(self.forward(X,hidden_layers)[0]).transpose(), layer_error))

        # there are hidden_layers + 1 number of weight matrices, hidden_layers - 1 if we calculate first and last one out of loop
        for step in range(2,hidden_layers+2):
            layer_error = np.array(self.forward(X, hidden_layers+2-step)) * (1 - np.array(self.forward(X, hidden_layers+2-step))) * np.dot(self.W[-step+1],np.matrix(layer_error).transpose())
            self.W[-step] += self.lr * (np.dot(np.matrix(self.forward(X,hidden_layers+1-step)[0]).transpose(), layer_error))


    # training: update the weights for the given number of epoches
    def train(self, X, y):
        for _ in range(self.number_of_epoches):
            self.backward(X, y)
    
    def total_error(self,X,y):
        return np.sum((y - self.forward(X))**2/2)  

In [105]:
import pandas as pd
data = pd.read_csv('housing.csv')
data.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


In [107]:
X = data[['RM','LSTAT','PTRATIO']]
y = data['MEDV']

In [111]:
np.array(data['RM'])

array([6.575, 6.421, 7.185, 6.998, 7.147, 6.43 , 6.012, 6.172, 5.631,
       6.004, 6.377, 6.009, 5.889, 5.949, 6.096, 5.834, 5.935, 5.99 ,
       5.456, 5.727, 5.57 , 5.965, 6.142, 5.813, 5.924, 5.599, 5.813,
       6.047, 6.495, 6.674, 5.713, 6.072, 5.95 , 5.701, 6.096, 5.933,
       5.841, 5.85 , 5.966, 6.595, 7.024, 6.77 , 6.169, 6.211, 6.069,
       5.682, 5.786, 6.03 , 5.399, 5.602, 5.963, 6.115, 6.511, 5.998,
       5.888, 7.249, 6.383, 6.816, 6.145, 5.927, 5.741, 5.966, 6.456,
       6.762, 7.104, 6.29 , 5.787, 5.878, 5.594, 5.885, 6.417, 5.961,
       6.065, 6.245, 6.273, 6.286, 6.279, 6.14 , 6.232, 5.874, 6.727,
       6.619, 6.302, 6.167, 6.389, 6.63 , 6.015, 6.121, 7.007, 7.079,
       6.417, 6.405, 6.442, 6.211, 6.249, 6.625, 6.163, 8.069, 7.82 ,
       7.416, 6.727, 6.781, 6.405, 6.137, 6.167, 5.851, 5.836, 6.127,
       6.474, 6.229, 6.195, 6.715, 5.913, 6.092, 6.254, 5.928, 6.176,
       6.021, 5.872, 5.731, 5.87 , 6.004, 5.961, 5.856, 5.879, 5.986,
       5.613, 5.693,